In [6]:
import numpy as np
import numba
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.layouts import gridplot
from bokeh.models import ColumnDataSource, HoverTool, CustomJS
output_notebook()



Loading BokehJS ...

In [30]:
#@numba.jit(nopython=True)
def ggov(t,mode,params):
    
    for it in range(len(params)):
    
        Dt = params[it].Dt
        Droop = params[it].Droop
        K_pgov = params[it].K_pgov
        K_igov = params[it].K_igov
        T_b = params[it].T_b
        T_c = params[it].T_c
        K_imw = params[it].K_imw 

        p_out_ref = params[it].p_out_ref
        p_out_meas = params[it].p_out_meas
        speed = params[it].speed
        gov_ref = params[it].gov_ref

        if mode == 0:
            x_turbine = params[it].p_mech

            params[it].x[2]  = x_turbine 
            params[it].p_out_ref = params[it].p_mech
            params[it].p_out_meas = params[it].p_mech
            params[it].gov_ref = 1.0 + Droop * params[it].p_mech
            params[it].x[1]  = 0.0

        x_lc  = params[it].x[0] 
        x_pid = params[it].x[1] 
        x_turbine = params[it].x[2]      

        gen_power = p_out_meas
        u_pid =  gov_ref - speed + x_lc - Droop * gen_power  
        u_turb = u_pid*K_pgov + x_pid

        if mode == 1: # der or update


            # Load controller
            dx_lc = K_imw * (p_out_ref - p_out_meas)

            # PID   
            dx_pid =  K_igov * (u_pid)

            # Lead-lag
            dx_turbine =  1/T_b*(u_turb-x_turbine)

            params[it].x[0] += Dt*dx_lc 
            params[it].x[1] += Dt*dx_pid 
            params[it].x[2] += Dt*dx_turbine 

        if mode == 2: # out

            p_mech = T_c/T_b*(u_turb-x_turbine)+x_turbine

            params[it].p_mech = p_mech         

#@numba.jit(nopython=True)
def gen(t,mode,params,params_gov):
    
    deltas = params.x[:,0,0]
    omegas = params.x[:,1,0]
    
    delta_coi = np.sum(deltas*params.H)/np.sum(params.H)
    omega_coi = np.sum(omegas*params.H)/np.sum(params.H)
    
    for it in range(len(params)):
        
        Dt = params[it].Dt
        H = params[it].H
        D = params[it].D
        p_load = params[it].p_load
        X_tie = params[it].X_tie
        Omega_b = params[it].Omega_b
        K_stab = params[it].K_stab
        
        
        delta = params[it].x[0]
        omega = params[it].x[1]
        
        p_m = -20.0*(omega - 1.0)
        
        #delta_coi = delta
        E = K_stab*(omega - omega_coi) + 1.0
        p_e = E*V/X_tie*np.sin(delta - delta_coi) + p_load
         
        params[it].E = E

        if mode == 0:
            ggov(t,0,params_gov)

            params[it].x[0]  = 0.0
            params[it].x[1]  = 1.0

        if mode == 1: # der or update

            params_gov[it].p_out_meas = p_e    
            params_gov[it].speed = omega 
            #ggov(t,1,params_gov)
            #ggov(t,2,params_gov)
            #p_m = params_gov[it].p_mech
            
            ddelta = Omega_b*(omega - omega_coi)
            domega = 1.0/(2*H)*(p_m - p_e  -  D*(omega-omega_coi))


            params[it].x[0] += Dt*ddelta 
            params[it].x[1] += Dt*domega 

        if mode == 2: # out

            params[it].p_e = p_e 
            params[it].p_m = p_m
            params[it].delta = params[it].x[0]  
            params[it].omega = params[it].x[1]  
            params[it].delta_coi = delta_coi
            params[it].omega_coi = omega_coi
            
            
def system(t,mode,params):
    
    it = 0
    
    H = params[it].H
    D = params[it].D
    Dfreq = params[it].Dfreq
    p_gen = params[it].p_gen
    p_load = params[it].p_load

    if mode == 0:
        
        params[it].x[0]  = 0.0
    
    if mode == 1: # der or update

        dDfreq = 1.0/(2*H)*(p_gen - p_load - D*Dfreq)

        params[it].f[0] = dDfreq 
        
        
    if mode == 2: # out
        
        params[it].Dfreq = params[it].x[0]     

In [37]:
dt_ggov = np.dtype([('Droop', np.float64),
                    ('K_pgov', np.float64),
                    ('K_igov', np.float64), 
                    ('T_b', np.float64), 
                    ('T_c', np.float64),
                    ('K_imw', np.float64),
                    ('f', np.float64, (3,1)),
                    ('x', np.float64, (3,1)),
                    ('p_out_ref', np.float64),
                    ('p_out_meas', np.float64),
                    ('speed', np.float64),
                    ('gov_ref', np.float64),
                    ('p_mech', np.float64),
                    ('Dt', np.float64) 
                   ]) 
    
Droop = 0.05
T_b = 10.0
T_c = 2.0
K_pgov = 10.0
K_igov = 2.0                   
K_imw = 0.0            

ggov_data  = [{"Droop":0.05,"T_b":10.0,"T_c":0.0, "K_pgov":10.0, "K_igov":2.0,"K_imw":0.0}]
ggov_data += [{"Droop":0.05,"T_b":2.0,"T_c":0.0, "K_pgov":10.0, "K_igov":2.0,"K_imw":0.0}]

ggov_list = []

for item in ggov_data:
    ggov_list += [(item['Droop'],
                   item['T_b'],
                   item['T_c'],
                   item['K_pgov'],
                   item['K_igov'],
                   item['K_imw'],
                        np.zeros((3,1)),
                        np.zeros((3,1)),
                        0.0,
                        0.0,
                        0.0,
                        0.0,
                        0.0,
                        0.01
                       )]
    
params_gov = np.rec.array(ggov_list, dtype=dt_ggov )

dt_sys = np.dtype([('H', np.float64),
                    ('D', np.float64),
                    ('Dfreq', np.float64),
                    ('p_gen', np.float64),
                    ('p_load', np.float64),
                    ('f', np.float64, (1,1)),
                    ('x', np.float64, (1,1))
                   ]) 
    
H = 5.0
D  = 2.0       
                   
params_sys = np.rec.array([(H,
                        D,
                        0.0,
                        0.0,
                        0.0,
                        np.zeros((1,1)),
                        np.zeros((1,1))
                       )], dtype=dt_sys )




dt_gen  = np.dtype([('Dt', np.float64),
                    ('H', np.float64),                    
                    ('D', np.float64),
                    ('K_stab', np.float64),
                    ('E', np.float64),
                    ('V', np.float64),
                    ('delta_coi', np.float64),
                    ('omega_coi', np.float64),                    
                    ('X_tie', np.float64),
                    ('Omega_b', np.float64),
                    ('omega', np.float64),
                    ('delta', np.float64),
                    ('p_m', np.float64),
                    ('p_e', np.float64),
                    ('p_load', np.float64),
                    ('f', np.float64, (2,1)),
                    ('x', np.float64, (2,1))
                   ]) 
    
H = 3.5
D  = 0.0
X_tie = 0.8
Omega_b = 2.0*np.pi*50.0
Dt = 0.01
E = 1.0
V = 1.0
K_stab = -10.0

gen_list = [(
            Dt,
            H, 
            D,
            K_stab,
            E,
            V,
            0.0,
            0.0,
            X_tie, #X_tie
            Omega_b,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,            
            np.zeros((2,1)),
            np.zeros((2,1))
           )] *2


params_gen = np.rec.array(gen_list, dtype=dt_gen )

In [38]:
N_g = len(params_gen)

#@numba.jit(nopython=True)
def run():
    N =5000
    T = np.zeros((N,1))
    out = np.zeros((12,N))
    Dt = 0.005
    gen(0.0,0,params_gen,params_gov)

    params_gen.Dt = Dt
    params_gov.Dt = Dt
    
    params_gen[0].x[0] = 0.1
    for it in range(N):
        t = it*Dt
        
        gen(0.0,1,params_gen,params_gov)
        gen(0.0,2,params_gen,params_gov)
        
        #params_gen[0].E = -200*(params_gen[0].omega - params_gen[1].omega) + 1.0
        #params_gen[1].E =  200*(params_gen[0].omega - params_gen[1].omega) + 1.0
        
        params_gen.p_load = 0.0
        if t>1.0: params_gen.p_load =0.2
        
        
        
        T[it,0] = t

        out[0,it]    = params_gen[0].delta
        out[1,it]    = params_gen[1].delta
        out[2,it]    = params_gen[0].omega
        out[3,it]    = params_gen[1].omega
        out[4,it]    = params_gen[0].p_e
        out[5,it]    = params_gen[1].p_e
        out[6,it]    = params_gen[0].p_m
        out[7,it]    = params_gen[1].p_m
        out[8,it]    = params_gen[0].E
        out[9,it]    = params_gen[1].E
        out[10,it]    = params_gen[0].omega_coi
        out[11,it]    = params_gen[1].omega_coi
    return T,out

T,out = run()

In [41]:
fig = figure(width=600, height=300)
fig.line(T[:,0],out[0,:],  line_width=1)
fig.line(T[:,0],out[1,:],  line_width=1, color='red')
show(fig)


In [40]:
fig = figure(width=600, height=300)
fig.line(T[:,0],out[4,:],  line_width=1)
fig.line(T[:,0],out[5,:],  line_width=1, color='red')
show(fig)


In [35]:
params_gen[0].E

1.0

In [438]:
        if t>0.1:
            params_sys[0].p_load = 1250/91e3

        for it_gov in range(len(params_gov)):
            params_gov[it_gov].speed = params_sys[0].Dfreq + 1.0 
            
        params_sys[0].x[:] += Dt*params_sys[0].f
        
        p_gen = 0.0
        for it_gov in range(len(params_gov)):
            p_gen += params_gov[it_gov].p_mech 
            #params_gov[it_gov].p_out_meas = params_sys[0].p_load/len(params_gov)
            
        #print(len(params_gov))
        params_gov[0].p_out_meas = params_sys[0].p_load  - params_gov[1].p_mech 
        params_gov[1].p_out_meas = params_sys[0].p_load  - params_gov[0].p_mech 
            
        params_sys[0].p_gen = p_gen
        
        
        T[it,0] = t
        out[0,it] =params_sys[0].Dfreq
        for it_gov in range(len(params_gov)):
            out[it_gov+1,it] =params_gov[it_gov].p_mech

NameError: name 't' is not defined

In [ ]:
for it_gov in range(len(params_gov)):
    params_gov[it_gov].p_mech = 0.0
    params_gov[it_gov].speed = 1.0        
        
ggov(0.0,0,params_gov)


def run():
    N = 3000
    T = np.zeros((N,1))
    out = np.zeros((12,N))
    Dt = 0.01
    ggov(0.0,0,params_gov)
    system(0.0,0,params_sys)

    for it_gov in range(len(params_gov)):
        params_gov[it_gov].Dt = Dt
    
    for it in range(N):
        t = it*Dt
        
        ggov(0.0,1,params_gov)
        ggov(0.0,2,params_gov)
        system(0.0,1,params_sys)
        system(0.0,2,params_sys)
        
        if t>0.1:
            params_sys[0].p_load = 1250/91e3

        for it_gov in range(len(params_gov)):
            params_gov[it_gov].speed = params_sys[0].Dfreq + 1.0 
            
        params_sys[0].x[:] += Dt*params_sys[0].f
        
        p_gen = 0.0
        for it_gov in range(len(params_gov)):
            p_gen += params_gov[it_gov].p_mech 
            #params_gov[it_gov].p_out_meas = params_sys[0].p_load/len(params_gov)
            
        #print(len(params_gov))
        params_gov[0].p_out_meas = params_sys[0].p_load  - params_gov[1].p_mech 
        params_gov[1].p_out_meas = params_sys[0].p_load  - params_gov[0].p_mech 
            
        params_sys[0].p_gen = p_gen
        
        
        T[it,0] = t
        out[0,it] =params_sys[0].Dfreq
        for it_gov in range(len(params_gov)):
            out[it_gov+1,it] =params_gov[it_gov].p_mech
        
    return T,out
        
        
        

In [ ]:
T, out = run()

In [ ]:
system(0.0,2,params_sys)
params_sys.p_load
params_sys.x

In [ ]:
fig = figure(width=600, height=300)
fig.line(T[:,0],(out[0,:]+1)*60,  line_width=1)
show(fig)
fig = figure(width=600, height=300)
fig.line(T[:,0],out[1,:],  line_width=1, color = 'red')
fig.line(T[:,0],out[2,:],  line_width=1, color = 'blue')
show(fig)